In [20]:
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander

## constants

In [21]:
group_number = 4
uri = f'radio://0/{group_number}/2M'
camera = 2
log_period_ms = 50

## testing camera

In [43]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     print(np.mean(hsv[210:270, 290:350, :], axis=(0, 1)))

    mask, largest_contour, area = detect_book()
    print(area)

    # Compute
    cv2.imshow('frame', mask)
#     cv2.imshow('frame', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()
del cap

0
12024.5
12245.0
12572.0
12949.5
12162.5
11305.5


QObject::moveToThread: Current thread (0x563b23262ce0) is not the object's thread (0x563b23833c60).
Cannot move to target thread (0x563b23262ce0)

QObject::moveToThread: Current thread (0x563b23262ce0) is not the object's thread (0x563b23833c60).
Cannot move to target thread (0x563b23262ce0)

QObject::moveToThread: Current thread (0x563b23262ce0) is not the object's thread (0x563b23833c60).
Cannot move to target thread (0x563b23262ce0)

QObject::moveToThread: Current thread (0x563b23262ce0) is not the object's thread (0x563b23833c60).
Cannot move to target thread (0x563b23262ce0)

QObject::moveToThread: Current thread (0x563b23262ce0) is not the object's thread (0x563b23833c60).
Cannot move to target thread (0x563b23262ce0)

QObject::moveToThread: Current thread (0x563b23262ce0) is not the object's thread (0x563b23833c60).
Cannot move to target thread (0x563b23262ce0)

QObject::moveToThread: Current thread (0x563b23262ce0) is not the object's thread (0x563b23833c60).
Cannot move to tar

10442.0
10877.5
10807.5
11196.5
11434.5
11529.0
11782.5
12689.5
11643.5
13094.0
12002.5
11377.0
12698.5
12402.5
11917.5
11949.5
12022.5
3357.5
3695.0
3746.0
12112.0
12359.5
12028.5
12261.5
12075.5
12910.0
13070.0
9151.5
8742.0
12389.0
13347.0
13172.0
13122.0
12856.5
12690.0
12606.0
12654.5
12514.5
12647.5
12783.5
12885.5
12836.0
12783.0
12848.0
12824.5
12899.5
12892.0
12997.5
12986.5
12980.0
12921.5
13022.0
12829.5
12867.5
12916.0
12880.5
12885.5
12771.0
12824.5
12718.5
12714.5
12625.0
12587.0
12418.5
12318.0
12229.0
11826.5
11983.5
11849.0
11957.5
11964.5
12024.5
12041.5
12140.0
12096.0
12097.5
12188.0
12156.0
12156.5
12058.5
9942.5
7692.0
10369.5
7765.5
9966.5
1474.0
8971.0
8743.0
7739.0
7575.0
6828.5
6730.0
6517.0
1504.0
3517.5
172.5
16.0
18.0
156.5
39.5
16.5
7.0
66.0
75.5
102.0
200.5
228.5
325.5
535.5
1123.5
1434.5
1532.0
1668.5
4822.0
4538.0
5128.0
5532.0
4929.0
4605.0
683.5
857.5
928.0
684.5
1627.0
2958.0
2754.0
597.0
602.0
784.5
908.0
1075.0
906.0
791.5
1106.5
887.0
1138.5
955.5

3967.5
120.5
1833.0
1845.5
3126.5
12728.0
4704.0
18183.0
15158.0
11089.5
4218.5
12805.5
12655.0
14495.5
12054.5
11013.5
2.0
26516.0
9906.0
13850.5
2740.5
10034.5
6820.5
10057.0
43.0
3255.5
7616.0
504.0
322.0
13675.5
6283.5
14839.0
9455.5
17563.5
17412.5
1432.0
21500.5
11028.5
10267.0
8393.5
9166.5
9275.5
75.5
28.0
47.5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10.0
14.0
25.5
0
0


## helper functions

In [44]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)
        
def ascend_and_hover(cf, height):
    print('ascending')
            
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, height * y / 10)
        time.sleep(0.1)

    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, height)
        time.sleep(0.1)
    
def hover_and_descend(cf, height):
    print('descending')
    
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, height)
        time.sleep(0.1)
    
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, height * (10 - y) / 10)
        time.sleep(0.1)
    
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
        
def position_callback(timestamp, data, log_conf):
    global x_cur, y_cur, z_cur
    x_cur = data['kalman.stateX']
    y_cur = data['kalman.stateY']
    z_cur = data['kalman.stateZ']
    
# Sort through contours in the image
def findGreatestContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index

def detect_obstacles():
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (30, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

#     # Compute
#     cv2.imshow('mask', mask)    

#     # Hit q to quit.
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    area, idx = findGreatestContour(contours)    
    largest_contour = contours[idx] if idx != -1 else None
    
    return mask, largest_contour, area

def detect_book():
    lower_blue = np.array([70,20,170])
    upper_blue = np.array([100,80,255])

    # Perform contour detection on the input frame.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of blue book in either color range.
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    area, idx = findGreatestContour(contours)
    largest_contour = contours[idx] if idx != -1 else None

    return mask, largest_contour, area

# loop

In [63]:
x_cur = 0.0
y_cur = 0.0
z_cur = 0.0

total_width = 1.6
left_boundary = total_width / 2
right_boundary = -total_width / 2

book_left_boundary = 0.7
book_right_boundary = -book_left_boundary

length = 96 * 0.0254
book_boundary = length
max_front_boundary = book_boundary + 0.15

obstacle_width = 3 * 0.0254 * 1.5

book_deviation = 0.2
obstacle_deviation = 0.1

min_obstacle_area = 5000
min_book_area = 1000

side_step = 0.08
book_side_step = 0.05
forward_step = 0.1
height = 0.9

go_left = True

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('FAILED! FAILED! FAILED!')
else:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf
        
        # set up logging
        log_conf = LogConfig(name='Position', period_in_ms=log_period_ms)
        log_conf.add_variable('kalman.stateX', 'float')
        log_conf.add_variable('kalman.stateY', 'float')
        log_conf.add_variable('kalman.stateZ', 'float')

        scf.cf.log.add_config(log_conf)
        log_conf.data_received_cb.add_callback(position_callback)
        log_conf.start()
        
        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False
        
        # start to hover
        set_PID_controller(cf)
        ascend_and_hover(cf, height)
        time.sleep(1)
        
        while cap.isOpened() and not exit_loop:
            ok, frame = cap.read()
            
            if not ok:
                print('no frame!')
                break
            
            print()
            mask, largest_contour, area = detect_obstacles()
            
            if x_cur > front_boundary:
                print(f"looking for book; x_cur = {x_cur:2f}; front_boundary = {front_boundary:.2f}")
                # look for book here
                mask, largest_contour, area = detect_book()
                
                if area < min_book_area:
                    # scan left to right
                    if y_cur > book_left_boundary: 
                        go_left = False
                    elif y_cur < book_right_boundary:
                        go_left = True
                    
                    if go_left:
                        print(f"no book found area = {area}; going left to {y_cur + book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur + book_side_step, height, 0)
                        time.sleep(0.1)
                    else:
                        print(f"no book found area = {area}; going right to {y_cur - book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur - book_side_step, height, 0)
                        time.sleep(0.1)
                    continue
                
                left_edge = np.amin(largest_contour[:,:,0])
                right_edge = np.amax(largest_contour[:,:,0])
                book_center = (left_edge + right_edge) / 2
                l_bound = mask.shape[1] * (0.5 - book_deviation)
                r_bound = mask.shape[1] * (0.5 + book_deviation)
                
                if book_center < l_bound:
                    # go right
                    print(f"book found area = {area}; book center = {book_center:.2f}; going right to {y_cur - book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                    cf.commander.send_position_setpoint(x_cur, y_cur - book_side_step, height, 0)
                    time.sleep(0.1)
                elif book_center > r_bound:
                    # go left
                    print(f"book found area = {area}; book center = {book_center:.2f}; going left to {y_cur + book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                    cf.commander.send_position_setpoint(x_cur, y_cur + book_side_step, height, 0)
                    time.sleep(0.1)
                else:
                    # land
                    if area < 4000 and x_cur < max_front_boundary:
                        print(f"book found area = {area}; book center = {book_center:.2f}; going forward to {x_cur + forward_step:2f}")
                        cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, height, 0)
                        time.sleep(0.1)
                    else:
                        print(f"book found area = {area}; book center = {book_center:.2f}; landing")
                        break
                
            elif area > min_obstacle_area:
                # adjust position
                left_edge = np.amin(largest_contour[:,:,0])
                right_edge = np.amax(largest_contour[:,:,0])
                
                
                l_bound = mask.shape[1] * (0.5 - obstacle_deviation)
                c_bound = mask.shape[1] * 0.5
                r_bound = mask.shape[1] * (0.5 + obstacle_deviation)
                
                print(f'looking for obstacle; area = {area}; width: {mask.shape[1]}, left edge: {left_edge}, right edge: {right_edge}')
                
                if right_edge < l_bound or left_edge > r_bound:
                    # go forward
                    print(f"contour not in bound; going forward to {x_cur + forward_step}, y_cur = {y_cur}")
                    cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, height, 0)
                    time.sleep(0.1)
                else:
                    mass_l = np.sum(mask[:, int(l_bound):int(c_bound)])
                    mass_r = np.sum(mask[:, int(c_bound):int(r_bound)])
                    print(f"contour in bound; left mass = {mass_l}, right mass = {mass_r}")
                    
                    if y_cur > left_boundary:
                        print(f"on the boundary, going right to {y_cur - obstacle_width:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur - obstacle_width, height, 0)
                        time.sleep(0.1)
                    elif y_cur < right_boundary:
                        print(f"on the boundary, going left to {y_cur + obstacle_width:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur + obstacle_width, height, 0)
                        time.sleep(0.1)
                    elif mass_l < mass_r:
                        # go left
                        print(f"going left to {y_cur + side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur + side_step, height, 0)
                        time.sleep(0.1)
                    else:
                        # go right
                        print(f"going right to {y_cur - side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur - side_step, height, 0)
                        time.sleep(0.1)
            else:
                # go forward
                print(f"small area = {area}; going forward to {x_cur + forward_step}, y_cur = {y_cur}")
                cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, height, 0)
                time.sleep(0.1)
            
#             mass_l = np.sum(mask[:, :mask.shape[1]//4])
#             mass_c = np.sum(mask[:, mask.shape[1]//4:mask.shape[1]*3//4])
#             mass_r = np.sum(mask[:, mask.shape[1]*3//4:])
#             print(mass_l, mass_c, mass_r)
            
        cap.release()
        
        # descend
        hover_and_descend(cf, height)

cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...


Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'

Crazyflies found:
radio://0/3/2M
radio://0/16/2M
radio://0/3/2M
radio://0/4/2M
radio://0/16/2M
radio://0/3/2M
radio://0/4/2M
radio://0/16/2M
radio://0/3/2M
radio://0/4/2M
radio://0/16/2M
radio://0/3/2M
radio://0/4/2M
radio://0/16/2M
radio://0/3/2M
radio://0/4/2M
radio://0/16/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
radio://0/3/2M
radio://0/4/2M
Initializing PID Controller
Waiting for estimator to find position...
999.9999896512181 999.9999898222959 999.9997560908814
999.9999906511466 999.9999903151456 999.9997560908814
999.9999906511466 999.9999903151456 999.9997595602617
999.9999906511466 999.9999903151456 999.9997595602617
999.9999906511466 999.9999903151456 999.9997595602617
999.9999906511466 999.9999903151456 999.99975956026


looking for obstacle; area = 21317.0; width: 640, left edge: 323, right edge: 396
contour in bound; left mass = 694110, right mass = 4555575
going left to -0.15, x_cur = 0.43, y_cur = -0.23

looking for obstacle; area = 20947.5; width: 640, left edge: 319, right edge: 397
contour in bound; left mass = 563805, right mass = 4117740
going left to -0.14, x_cur = 0.44, y_cur = -0.22

looking for obstacle; area = 22321.5; width: 640, left edge: 324, right edge: 400
contour in bound; left mass = 527085, right mass = 4359480
going left to -0.13, x_cur = 0.44, y_cur = -0.21

looking for obstacle; area = 24790.0; width: 640, left edge: 315, right edge: 411
contour in bound; left mass = 467670, right mass = 4905945
going left to -0.11, x_cur = 0.44, y_cur = -0.19

looking for obstacle; area = 10807.0; width: 640, left edge: 351, right edge: 398
contour in bound; left mass = 128520, right mass = 3376200
going left to -0.10, x_cur = 0.43, y_cur = -0.18

looking for obstacle; area = 11246.0; width:


looking for obstacle; area = 20588.5; width: 640, left edge: 546, right edge: 639
contour not in bound; going forward to 1.1910327434539796, y_cur = 0.03090151771903038

looking for obstacle; area = 15508.0; width: 640, left edge: 163, right edge: 235
contour not in bound; going forward to 1.2113023757934571, y_cur = 0.02866818569600582

looking for obstacle; area = 16898.5; width: 640, left edge: 156, right edge: 234
contour not in bound; going forward to 1.2277152299880982, y_cur = 0.029044074937701225

looking for obstacle; area = 16112.5; width: 640, left edge: 153, right edge: 235
contour not in bound; going forward to 1.2518917083740235, y_cur = 0.032338209450244904

looking for obstacle; area = 17324.5; width: 640, left edge: 149, right edge: 230
contour not in bound; going forward to 1.2690653562545777, y_cur = 0.03348241373896599

looking for obstacle; area = 15306.5; width: 640, left edge: 146, right edge: 230
contour not in bound; going forward to 1.2921147108078004, y_cur 


looking for obstacle; area = 7565.0; width: 640, left edge: 452, right edge: 639
contour not in bound; going forward to 2.235399103164673, y_cur = 0.17827878892421722

small area = 167.0; going forward to 2.252071237564087, y_cur = 0.17949923872947693

small area = 353.0; going forward to 2.268126344680786, y_cur = 0.17966040968894958

small area = 825.5; going forward to 2.2945786476135255, y_cur = 0.181923046708107

looking for obstacle; area = 8389.5; width: 640, left edge: 94, right edge: 247
contour not in bound; going forward to 2.316552734375, y_cur = 0.17932826280593872

looking for obstacle; area = 10149.5; width: 640, left edge: 0, right edge: 116
contour not in bound; going forward to 2.3372727394104005, y_cur = 0.17945709824562073

small area = 1088.5; going forward to 2.3476723194122315, y_cur = 0.17995530366897583

small area = 594.0; going forward to 2.366820192337036, y_cur = 0.18213556706905365

small area = 1302.0; going forward to 2.392210340499878, y_cur = 0.180228

In [62]:
cap.release()